## _*Chemistry with Qiskit Aqua and t| ket>*_ 

<!---
The latest version of this notebook is available on https://github.com/qiskit/qiskit-tutorial.

***
#### Contributors
Jay Gambetta[1], Ismael Faro[1], Andrew Cross[1], Ali Javadi[1]

#### Affiliation
* [1] IBM Q* 
--->

This notebook demonstrates calculation of excited states of molecules using quantum algorithms. The example shows Qiskit Aqua interface combined with fast circuit compilation and optimization via t|ket>. 

The example makes use of the Quantum Subspace Expansion [1, 2] as implemented in pytket (the python interface to t|ket>). 

In order to perform this calculation, we start from a ground state wavefunction $\Psi_0$ calculated with the Variational Quantum Eigensolver (VQE) technique. We construct a subspace of state vectors $\Psi_i^k$ formed by one-electron excitations of the ground state wavefunction:

\begin{equation}
|\Psi_i^k> = c_k^{\dagger}c_i|\Psi_0>
\end{equation}

where $c_k^{\dagger}$, $c_i$ are the fermionic creation and annihilation operators over spinorbitals k and i, respectively. The fermionic operators act by reducing the occupation of spinorbital i to zero, and increasing the occupation of spinorbital k to one. Then we solve the generalized eigenvalue problem corresponding to this subspace, whose equation is:

\begin{equation}
HC=SCE
\end{equation}

where $C$ is the matrix of eigenvectors, $E$ is the vector of eigenvalues, $S$ is the overlap matrix and $H$ is the Hamiltonian matrix for the subspace considered. The elements h$_{ij}^{kl}$ of $H$ are of the form:

\begin{equation}
h_{ij}^{kl} =  <\Psi_i^k | \widehat{H} | \Psi_j^l>
\end{equation}

Where $\widehat{H}$ is the molecular hamiltonian operator. In a similar way, the elements s$_{ij}^{kl}$ of $S$ are found from the following expression:

\begin{equation}
s_{ij}^{kl} = <\Psi_i^k | \Psi_j^l>
\end{equation}

The values for these matrix elements are calculated via the quantum computer.


Let’s illustrate this method with a simple example, the lithium hydride (LiH) molecule. First let's start by importing qiskit and pytket.

In [1]:
import qiskit
import pytket

## Configure experiment <a id='sectionB'></a>

We choose the basis set from which we will span the molecular wavefunction, the backend that will execute the quantum algorithm, the classical optimizer for the VQE routine and the quantum chemistry driver that will calculate the interelectronic integrals and the set of molecular orbitals needed for our calculation. We also set up the molecular geometry, the chemical identity of each atom, the charge and spin quantum number.

If you would like to run the experiement on a real device, you need to setup your account first.
Note: If you do not store your token yet, use IBMQ.save_accounts() to store it first.

In [2]:
bond_length = 0.7

# setup molecule
# base_molecule_str = 'Li .0 .0 .0; H .0 .0 {}'
base_molecule_str = 'H .0 .0 .0; H .0 .0 {}'
charge = 0
spin = 0
basis = 'sto3g'

# choose optimizer
optimizer_dict = {
    'name': 'L_BFGS_B',
    'options': {'maxfun':1000, 'factr':10, 'iprint':10}
}

In [3]:
# choose quantum backend (e.g. 'statevector_simulator', 'qasm_simulator', 'ibmqx5')
# for a remote backend you will need to load your API account

# local backends
backend_name = 'statevector_simulator'
backend = qiskit.Aer.get_backend(backend_name)

# remote backends
# qiskit.IBMQ.load_accounts()
# backend_name = 'ibmqx5'
# backend = qiskit.IBMQ.get_backend(backend)

Setup quantum instance with t|ket> compiler pass.

In [4]:
from qiskit_aqua import QuantumInstance
from qiskit.transpiler._passmanager import PassManager
from pytket.qiskit import TketPass

pass_manager = PassManager()
tk_pass = TketPass(backend)
pass_manager.add_passes(tk_pass)

quantum_instance = QuantumInstance(backend, pass_manager=pass_manager)

## Perform classical calculation <a id='sectionB'></a>

We execute our driver to obtain the hamiltonian integrals. In this case, we choose PYSCF as our driver, so make sure you have that installed.


In [5]:
from qiskit_aqua_chemistry.drivers import ConfigurationManager

# using driver to get fermionic Hamiltonian
# PySCF example
cfg_mgr = ConfigurationManager()
driver = cfg_mgr.get_driver_instance('PYSCF')

pyscf_cfg = {'atom': base_molecule_str.format(bond_length),
             'unit': 'Angstrom',
             'charge': charge,
             'spin': spin,
             'basis': basis}

molecule = driver.run({'properties': pyscf_cfg})

print("Molecular repulsion energy: ", molecule._nuclear_repulsion_energy)

Molecular repulsion energy:  0.7559674441714287


## Set up initial state and variational form <a id='sectionB'></a>

We define the molecular hamiltonian and map it to a qubit operator according to some appropriate scheme, like for example the Jordan-Wigner transform. We initialize our initial state (a Hartree-Fock state) and our variational ansatz (unitary coupled cluster with single and double excitations, UCCSD).

In [6]:
from qiskit_aqua_chemistry import FermionicOperator
from qiskit_aqua_chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit_aqua_chemistry.aqua_extensions.components.variational_forms import UCCSD

n_qubits = molecule._one_body_integrals.shape[0]
n_electrons = molecule._num_alpha + molecule._num_beta - molecule._molecular_charge

# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule._one_body_integrals, h2=molecule._two_body_integrals)

qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
n_qubits = qubitOp.num_qubits
qubitOp.chop(10**-10)

# setup variation form generator (generate trial circuits for VQE)
initial_hf = HartreeFock(num_qubits=n_qubits, num_orbitals=n_qubits, 
                    qubit_mapping='jordan_wigner', two_qubit_reduction=False, num_particles= n_electrons)

var_form = UCCSD(num_qubits=n_qubits, num_orbitals=n_qubits, 
                num_particles=n_electrons, depth=1, initial_state=initial_hf, qubit_mapping='jordan_wigner')



## Use VQE to find ground state <a id='sectionB'></a>

We run the VQE algorithm with the components previously defined. The calculation provides us with the ground state energy and the optimized parameters of the corresponding UCCSD wavefunction. In this example we use the L_BFGS_B algorithm for classical optimization.

In [7]:
from qiskit_aqua.algorithms.components.optimizers import L_BFGS_B
from qiskit_aqua.algorithms.adaptive import VQE

number_amplitudes = len(var_form._single_excitations)+ len(var_form._double_excitations)

amplitudes_0 = []
for i in range(number_amplitudes):
    amplitudes_0.append(0.00001)

optimizer = L_BFGS_B()
optimizer.set_options(maxfun=1000, factr=10, iprint=10)

# setup VQE_P with operator, variation form, and optimzer
vqe_algorithm = VQE(operator=qubitOp, operator_mode='matrix', 
                    var_form=var_form, optimizer=optimizer, initial_point=amplitudes_0)


# vqe_algorithm.setup_quantum_backend(backend=backend_name, pass_manager=pass_manager)
results = vqe_algorithm.run(quantum_instance)

eigval = results['eigvals'][0]
gs_energy = eigval.real + molecule._nuclear_repulsion_energy

print("GS Minimum value: {}".format(gs_energy))
print("GS Parameters: {}".format(results['opt_params']))

# store ground state amplitudes for subsequent steps
opti_amplitudes = results['opt_params']



GS Minimum value: -1.1361894540654025
GS Parameters: [ 5.05895911e-07  5.05922777e-07 -1.04867306e-01]


## Use QSE to find excited states from ground state <a id='sectionB'></a>

We now have the main ingredients to perform a QSE calculation: the molecular hamiltonian and the optimized parameters to reconstruct the ground state wavefunction. We build our excitation hamiltonian and overlap operators, and measure the elements that compose $H$ and $S$. After we have obtained these arrays, we perform a diagonalization to obtain the excited state energies and vectors.


In [8]:
from pytket.chemistry import QseMatrices, QSE

qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
n_qubits = qubitOp.num_qubits
qubitOp.chop(10**-10)
# use matrix term helper class
matrix_terms = QseMatrices(qubitOp, n_qubits)

qse_algorithm = QSE(matrix_terms, 'matrix', var_form, opt_init_point=opti_amplitudes)

energies = qse_algorithm.run(quantum_instance)['eigvals']
print("Excited State Energies: ", energies+molecule._nuclear_repulsion_energy)



Excited State Energies:  [-1.13618945 -0.47845306 -0.47845306 -0.47845306 -0.1204519   0.5833141
  0.75596744  0.75596744  0.75596744  0.75596744  0.75596744  0.75596744
  0.75596744  0.75596744  0.75596744  0.75596744]


The calculation provides a refined value of the ground state and a series of excited state energies, whose number depends on the size of the basis set chosen for the molecule, as well as its nature and symmetry. Some of the energies obtained are repeated several times, signaling that some of the states obtained are degenerate. This result can be improved by either increasing the basis set or considering higher-order excitations in the subspace expansion.

[1] J. R. McClean, M. E. Kimchi-Schwartz, J. Carter and W. A. de Jong, Phys. Rev. A 95, 042308 (2017)

[2] J. I. Colless, V. V. Ramasesh, D. Dahlen, M. S. Blok, M. E. Kimchi-Schwartz, J. R. McClean, J. Carter, W. A. de Jong and I. Siddiqi, Phys. Rev. X 8, 011021 (2018)
